In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import *
from db import *
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse

In [ ]:
with open('annotations_avatar_name_info.yaml', 'r') as fd:
    avatar_to_anns = { name: Bunch(name=name, **anns) for name,anns in yaml.safe_load(fd).items() }

with open('annotations_aid_avatars.yaml', 'r') as fd:
    aid_to_anns = { aid: Bunch(avatar_id=aid, **merge_anns(anns, avatar_to_anns.get(anns.get('name', None), {})))
                   for aid, anns in yaml.safe_load(fd).items()
                   }

len(avatar_to_anns), next(iter(avatar_to_anns.items())), len(aid_to_anns), next(iter(aid_to_anns.items()))

In [ ]:
with gzip.open('py_dataset_ms.json.gz', 'r') as f:
    ms = load_ms()

msd = defaultdict(list)
for m in ms:
    if get_in(m, 'targetPlayer') is None or get_in(m, 'extra.targetPlayerId') is None: continue
    aid = makeup_avatar_id( m )
    msd[ aid ].append( m )

len(ms), len(msd)

In [ ]:
for aid, ms_ in msd.items():
    m = ms_to_most_facing( ms_ )
    a = m.targetPlayer.avatar
    extras = AttrDict(
        most_facing = m.f,
        upload_name = a.name,
        uploader_id = a.authorId
    )
    anns = AttrDict(aid_to_anns.get(aid) or {})
    anns.dataset = 'daky'
    anns.extras = extras
    aid_to_anns[ aid ] = anns


In [ ]:
next(itertools.islice(iter(aid_to_anns.items()), 500, None))

In [ ]:
class AviDB(SQLiteDB):

  def __init__(self, path='avi.db'):
    col_types = [
      ('avatar_id', 'text'),
      ('name', 'text'),
      ('creator', 'text'),
      ('url', 'text'),
      ('category', 'text'),
      ('dataset', 'text'),
      ('extras', 'json')
    ]
    name = 'avi'
    super().__init__(path=path, name=name, col_types=col_types)
    
  def _create_table(self, col_types):
    self.execute('CREATE VIRTUAL TABLE IF NOT EXISTS %s USING fts4(%s)' % (self.name, ', '.join(['%s %s %s' % (c, t, 'UNIQUE' if i == 0 else '') for i, (c, t) in enumerate(col_types)])))
    
  def _create_index(self):
    pass

In [ ]:
avi_db = AviDB()

In [ ]:
avi_db.insert(aid_to_anns.values())

In [ ]:
avi_db.size()

In [ ]:
for avi in avi_db.iter_rows():
  print(avi)

In [ ]:
avi_db.get(2)

In [ ]:
avi_db.get_aid('usr_0044f8fa-3e35-41a6-af75-87dc40a975f3:Corpse Candy')

In [ ]:
list(avi_db.match(name='Rusk'))

In [ ]:
list(avi_db.search(creator='Kyubihome', category='jp'))

In [ ]:
list(avi_db.search(name='%hime%', op='LIKE'))